# Probabilistic SBB CFF Journey Planner

## Introduction

The objective of this project is to implement a robust journey planner for the SBB CFF public transport network, with a specific focus on the Zürich area. As opposed to the already exisiting journey planner offered by SBB, we develop a journey planner that takes into consideration the historical punctuality or delays of trips when these are suggested to the user as part of a journey. In other words, the planner we develop computes the probability of arriving within a time window when suggesting a journey based on the historical punctuality of the trips that are part of it. For journeys that are composed of multiple trips, the computed probability also illustrates the chance of successfully making all of the suggested connections.

At the start of the project, we were provided with the the Swiss Public Transport data that is made available at:
https://opentransportdata.swiss/en/

In order to create the mentioned planner from the given data, we divided our approach in the following steps:
- Data pre-processing and cleaning
- Modelling the public transport network
- Creating the journey planner algorithm
- Visualization of results

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from math import radians, cos, sin, asin, sqrt
from pyspark.sql.functions import lag, col
from pyspark.sql.window import Window
import pyspark.sql.functions as functions
from pyspark.sql.functions import when
from pyspark.sql.types import IntegerType
import time

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import os
import getpass

username = getpass.getuser()

os.environ['PYSPARK_PYTHON'] = '/opt/anaconda3/bin/python'
spark = (SparkSession
         .builder
         .appName('streaming-{0}'.format(username))
         .master('yarn')
         .config('spark.executor.memory', '4g')
         .config('spark.executor.instances', '4')
         .config('spark.executor.cores', '4')
         .config('spark.port.maxRetries', '100')
         .getOrCreate())

sc = spark.sparkContext
conf = sc.getConf()

spark

## l. Data Pre-processing & Cleaning

### A. Schedule Data

We decided to mainly work with the 2019 data in order to obtain the schedule data for this year. We avoided the use of 2018 and 2017 data as schedules are changed on a yearly basis at the end of each year. We opted with loading the data from the provided datasets on the HDFS as this made it easy to directly leverage spark.

In [3]:
df_2019 = spark.read.csv('/datasets/sbb/2019/*/*.bz2', sep=';', header=True)

In [4]:
df_2019 = df_2019.selectExpr("BETRIEBSTAG as date", 
                             "FAHRT_BEZEICHNER as trip_id",
                             "BETREIBER_ID as operator_id",
                             "BETREIBER_ABK as operator_ab",
                             "BETREIBER_NAME as operator_name", 
                             "PRODUKT_ID as transport_type",
                             "LINIEN_ID as train_id", 
                             "LINIEN_TEXT as train_type", 
                             "UMLAUF_ID as circulation_id",
                             "VERKEHRSMITTEL_TEXT as train_type2", 
                             "ZUSATZFAHRT_TF as additional_trip",
                             "FAELLT_AUS_TF as trip_failed", 
                             "BPUIC as bpuic", 
                             "HALTESTELLEN_NAME as stop_name",
                             "ANKUNFTSZEIT as arrival_schedule", 
                             "ABFAHRTSZEIT as departure_schedule",
                             "AN_PROGNOSE as arrival_actual",
                             "AB_PROGNOSE as departure_actual",
                             "AN_PROGNOSE_STATUS as arrival_status",
                             "AB_PROGNOSE_STATUS as departure_status",
                             "DURCHFAHRT_TF as does_not_stop")

As our focus for this project was the Zürich area, we imported the geolocation data for every stop in order to subsequently filter out every stop that falls outside of this zone (10km radius from Zürich HB). This data is also made available on: https://opentransportdata.swiss/en/

In [5]:
geo = open("BFKOORD_GEO", "r", encoding = 'utf-8-sig')
geo = geo.readlines()

In [6]:
all_stops = []
for line in geo:
    stop = []
    stop.append(int(line[0:7]))
    stop.append(float(line[9:18]))
    stop.append(float(line[20:29]))
    stop.append(int(line[30:].split('%')[0].strip()))
    stop.append(line[30:].split('%')[1][1:-1])
    all_stops.append(stop)

In [7]:
def haversine(lon1, lat1, lon2, lat2):
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers
    return c * r

In [8]:
all_stops_pd = pd.DataFrame(all_stops, columns=['index', 'long', 'lat', 'num', 'stop'])
# hardcoded Zürich HB coordinates
all_stops_pd['from_ZHB'] = np.vectorize(haversine)(8.540192, 47.378177, all_stops_pd.long, all_stops_pd.lat)
zurich_area_stops = all_stops_pd[all_stops_pd.from_ZHB <=10]

In [9]:
zurich_area_stops.head()

,index,long,lat,num,stop,from_ZHB
74,176,8.521961,47.351679,0,Zimmerberg-Basistunnel,3.250670
2084,8502220,8.434713,47.390882,442,Urdorf,8.065908
2085,8502221,8.437543,47.357432,488,Birmensdorf ZH,8.067431
2086,8502222,8.468175,47.325896,528,Bonstetten-Wettswil,7.951687
2092,8502229,8.430330,47.380971,456,Urdorf Weihermatt,8.277819


Having the list of stops that are within 10 kilometers of the Zürich HB train stations, we now filter the 2019 SBB CFF data to these stops.

In [10]:
inscope_stops = list(zurich_area_stops.stop.values)
zurich_df = df_2019.filter(df_2019.stop_name.isin(inscope_stops))

The raw data we have now represents the stops within the Zürich area that have been made in January 2019 by any type of public transport. Each stop is associated with a number of data points including: the schedule date, the arrival/departure times according to schedule, a trip id that identifies the trip within a single day and many others. As a first step, we remove those trips that were not part of the normal schedule (additional trips), the stops where the trip did not stop and those trips that failed.

In [11]:
schedule_df = zurich_df.filter((zurich_df.additional_trip == 'false') & (zurich_df.does_not_stop == 'false')
                               & (zurich_df.trip_failed == 'false') & (zurich_df.train_type2 != 'SN') 
                               & (zurich_df.train_type2 != 'BN'))

Throughout this project, we decided to work with the schedule times represented as the number of minutes from midnight of the relevant schedule day. It is important to note that certain trips that are part of one day's schedule, actually occur the next day. For example, a bus can be part of the Wednesday schedule but travels on Thursday as its journey takes place after midnight. Hence, in this case its schedule arrival/departure times will be greater than 60 x 24 (1440) as it represents the number of minutes from the start of its schedule day. As such, we created a column indicating whether a trip occurs on the day following its associated schedule day.

In [12]:
# keep essential columns
schedule_df = schedule_df.select(['date', 'trip_id', 'stop_name', 'arrival_schedule', 
                                  'departure_schedule'])

In [13]:
# separate the date and time information from the arrival/departure schedule column into two separate columns
schedule_df = schedule_df.withColumn('arrival_date_schedule', schedule_df.arrival_schedule[0:10])
schedule_df = schedule_df.withColumn('arrival_time_schedule', schedule_df.arrival_schedule[12:15])
schedule_df = schedule_df.withColumn('departure_date_schedule', schedule_df.departure_schedule[0:10])
schedule_df = schedule_df.withColumn('departure_time_schedule', schedule_df.departure_schedule[12:15])

In [14]:
@functions.udf
def night_travel(date, test_date, failsafe_date):
    d = date[:2]
    if test_date is None:
        fd = failsafe_date[:2]
        return int(fd) - int(d)
    td = test_date[:2]
    return int(td) - int(d)

In [15]:
# indicates whether the arrival takes place the next day
schedule_df = schedule_df.withColumn('arrival_next_day', night_travel(schedule_df.date, 
                                                                      schedule_df.arrival_date_schedule, 
                                                                      schedule_df.departure_date_schedule))

# indicates whether the departure takes place the next day
schedule_df = schedule_df.withColumn('departure_next_day', night_travel(schedule_df.date, 
                                                                        schedule_df.departure_date_schedule,
                                                                        schedule_df.arrival_date_schedule))

In [16]:
schedule_df = schedule_df.withColumn("arrival_next_day", schedule_df["arrival_next_day"].cast(IntegerType()))
schedule_df = schedule_df.withColumn("departure_next_day", schedule_df["departure_next_day"].cast(IntegerType()))

In order to extract accurate representations of the schedules, we decided to split the schedule data between week days, saturdays and sundays. This is because the schedule changes between these three. Note that we selected one day for each schedule as we noticed a trip could have different trip ids on different days.

In [17]:
# separate saturdays, sundays and week days into separate dataframes
saturday_df = schedule_df.filter(schedule_df.date == '19.01.2019')
sunday_df = schedule_df.filter(schedule_df.date == '20.01.2019')
weekday_df = schedule_df.filter(schedule_df.date == '22.01.2019')

In [18]:
# remove date information from each dataframe
saturday_df = saturday_df.select('trip_id', 'stop_name', 'arrival_time_schedule', 'departure_time_schedule', 
                                 'arrival_next_day', 'departure_next_day')

sunday_df = sunday_df.select('trip_id', 'stop_name', 'arrival_time_schedule', 'departure_time_schedule', 
                             'arrival_next_day', 'departure_next_day')

weekday_df = weekday_df.select('trip_id', 'stop_name', 'arrival_time_schedule', 'departure_time_schedule', 
                               'arrival_next_day', 'departure_next_day')

In [19]:
# after dropping date column, remove duplicates (which in theory should narrow down to the schedule)
saturday_df = saturday_df.distinct()
sunday_df = sunday_df.distinct()
weekday_df = weekday_df.distinct()

As previously mentioned, we convert the scheduled arrival and departure times to the number of minutes from midnight of the associated schedule day. In the event where the arrival time is null, we replace it with the stops departure time and vice versa. This removes the nulls in both columns as there are no instances where both schedule times are null.

In [20]:
@functions.udf
def to_min_from_midnight(target_time, target_nd, backup_time, backup_nd):
    if target_time is None:
        t = backup_time.split(":")
        mins = int(t[0]) * 60 + int(t[1])
        if backup_nd == 0:
            return mins
        else:
            return mins + 60 * 24
    else:
        t = target_time.split(":")
        mins = int(t[0]) * 60 + int(t[1])
        if target_nd == 0:
            return mins
        else:
            return mins + 60 * 24


saturday_df = saturday_df.withColumn("arrival_time_schedule_m", \
                                     to_min_from_midnight(saturday_df.arrival_time_schedule,\
                                                          saturday_df.arrival_next_day, \
                                                          saturday_df.departure_time_schedule, \
                                                          saturday_df.departure_next_day))

saturday_df = saturday_df.withColumn("departure_time_schedule_m", \
                                     to_min_from_midnight(saturday_df.departure_time_schedule, \
                                                          saturday_df.departure_next_day, \
                                                          saturday_df.arrival_time_schedule, \
                                                          saturday_df.arrival_next_day))

sunday_df = sunday_df.withColumn("arrival_time_schedule_m", \
                                     to_min_from_midnight(sunday_df.arrival_time_schedule,\
                                                          sunday_df.arrival_next_day, \
                                                          sunday_df.departure_time_schedule, \
                                                          sunday_df.departure_next_day))

sunday_df = sunday_df.withColumn("departure_time_schedule_m", \
                                     to_min_from_midnight(sunday_df.departure_time_schedule, \
                                                          sunday_df.departure_next_day, \
                                                          sunday_df.arrival_time_schedule, \
                                                          sunday_df.arrival_next_day))

weekday_df = weekday_df.withColumn("arrival_time_schedule_m", \
                                     to_min_from_midnight(weekday_df.arrival_time_schedule,\
                                                          weekday_df.arrival_next_day, \
                                                          weekday_df.departure_time_schedule, \
                                                          weekday_df.departure_next_day))

weekday_df = weekday_df.withColumn("departure_time_schedule_m", \
                                     to_min_from_midnight(weekday_df.departure_time_schedule, \
                                                          weekday_df.departure_next_day, \
                                                          weekday_df.arrival_time_schedule, \
                                                          weekday_df.arrival_next_day))

In [21]:
saturday_df = saturday_df.selectExpr("trip_id",
                                     "stop_name",
                                     "arrival_time_schedule_m as arrival_time_schedule",
                                     "departure_time_schedule_m as departure_time_schedule")

sunday_df = sunday_df.selectExpr("trip_id",
                                 "stop_name",
                                 "arrival_time_schedule_m as arrival_time_schedule", 
                                 "departure_time_schedule_m as departure_time_schedule")

weekday_df = weekday_df.selectExpr("trip_id",
                                   "stop_name",
                                   "arrival_time_schedule_m as arrival_time_schedule", 
                                   "departure_time_schedule_m as departure_time_schedule")

During the pre-processing of the data, we noticed on a given day a stop could potentially have erroneous data in its scheduled departure or arrival times. After some experiments, we observed that taking the minimum schedule arrival time and the maximum schedule departure time for a given stop from a given trip provided positive results.

In [22]:
saturday_df = saturday_df.groupBy('trip_id', 'stop_name') \
                            .agg(functions.min(col('arrival_time_schedule')).alias('arrival_time_schedule'),
                                 functions.max(col('departure_time_schedule')).alias('departure_time_schedule'))

In [23]:
sunday_df = sunday_df.groupBy('trip_id', 'stop_name') \
                            .agg(functions.min(col('arrival_time_schedule')).alias('arrival_time_schedule'),
                                 functions.max(col('departure_time_schedule')).alias('departure_time_schedule'))

In [24]:
weekday_df = weekday_df.groupBy('trip_id', 'stop_name') \
                            .agg(functions.min(col('arrival_time_schedule')).alias('arrival_time_schedule'),
                                 functions.max(col('departure_time_schedule')).alias('departure_time_schedule'))

Finally, we observed that in our schedule data there are singleton stops. In other words, there were stops that are part of long distance trips which only stop once within the Zürich area. These are of no use for our journey planner as they do not connect two stops within the area of focus. As such, we removed them for the saturday, sunday and week day schedules.

In [25]:
line_count_window = Window.partitionBy('trip_id')
line_stop_count = functions.count('trip_id') \
                      .over(line_count_window) \
                      .alias('stop_count')

saturday_df = saturday_df.select('trip_id', 'stop_name', 'arrival_time_schedule', 'departure_time_schedule', 
                                 line_stop_count)

sunday_df = sunday_df.select('trip_id', 'stop_name', 'arrival_time_schedule', 'departure_time_schedule', 
                             line_stop_count)

weekday_df = weekday_df.select('trip_id', 'stop_name', 'arrival_time_schedule', 'departure_time_schedule', 
                               line_stop_count)

saturday_df = saturday_df.filter(saturday_df.stop_count>1)
sunday_df = sunday_df.filter(sunday_df.stop_count>1)
weekday_df = weekday_df.filter(weekday_df.stop_count>1)

In [26]:
import pickle
import time

begin = time.time()
LOAD_FROM_MEM = True
if not LOAD_FROM_MEM:
    saturday_pd = saturday_df.toPandas()
    pickle.dump(saturday_pd, open("./graph_saved/saturday_pd.pickle", "wb"))
    weekday_pd = weekday_df.toPandas()
    pickle.dump(weekday_pd, open("./graph_saved/weekday_pd.pickle", "wb"))
    sunday_pd = sunday_df.toPandas()
    pickle.dump(sunday_pd, open("./graph_saved/sunday_pd.pickle", "wb"))
else:
    saturday_pd = pickle.load(open("./graph_saved/saturday_pd.pickle", "rb"))
    weekday_pd = pickle.load(open("./graph_saved/weekday_pd.pickle", "rb"))
    sunday_pd = pickle.load(open("./graph_saved/sunday_pd.pickle", "rb"))

print("Time taken:", time.time()-begin)

Time taken: 0.5702521800994873


### B. Historic Data

For the historic data relating to the punctuality of trips, we obtained all of the data that was made available on the HDFS. This includes data from 2017, 2018 and 2019.

In [27]:
dist_df = spark.read.csv('/datasets/sbb/*/*/*.bz2', sep=';', header=True)

In [28]:
dist_df = dist_df.selectExpr("BETRIEBSTAG as date", 
                             "FAHRT_BEZEICHNER as trip_id",
                             "BETREIBER_ID as operator_id",
                             "BETREIBER_ABK as operator_ab",
                             "BETREIBER_NAME as operator_name", 
                             "PRODUKT_ID as transport_type",
                             "LINIEN_ID as train_id", 
                             "LINIEN_TEXT as train_type", 
                             "UMLAUF_ID as circulation_id",
                             "VERKEHRSMITTEL_TEXT as train_type2", 
                             "ZUSATZFAHRT_TF as additional_trip",
                             "FAELLT_AUS_TF as trip_failed", 
                             "BPUIC as bpuic", 
                             "HALTESTELLEN_NAME as stop_name",
                             "ANKUNFTSZEIT as arrival_schedule", 
                             "ABFAHRTSZEIT as departure_schedule",
                             "AN_PROGNOSE as arrival_actual",
                             "AB_PROGNOSE as departure_actual",
                             "AN_PROGNOSE_STATUS as arrival_status",
                             "AB_PROGNOSE_STATUS as departure_status",
                             "DURCHFAHRT_TF as does_not_stop")

We applied the same filtering conditions to the historic data as the ones applied earlier to the schedule data. Additionally, we also compute the day of the week for each row as we separate the historic data between saturdays, sundays and week days. Finally, we also remove trips that only have one stop within the Zürich area.

In [29]:
dist_df = dist_df.filter(dist_df.stop_name.isin(inscope_stops))
dist_df = dist_df.filter((dist_df.additional_trip == 'false')
                         & (dist_df.does_not_stop == 'false')
                         & (dist_df.trip_failed == 'false'))

In [30]:
dist_df = dist_df.withColumn('day_of_week', 
                               functions.dayofweek(
                                   functions.from_unixtime(
                                       functions.unix_timestamp('date', 'dd.MM.yyy'))))

In [31]:
line_day_window = Window.partitionBy('trip_id', 'date')

line_day_stop_count = functions.count('stop_name') \
                    .over(line_day_window) \
                    .alias('stop_count')

In [32]:
from pyspark.sql.functions import stddev, avg

# Truncates a decimal number
def truncate(n, decimals=0):
    multiplier = 10 ** decimals
    return int(n * multiplier) / multiplier

def to_sec_from_midnight(hour_min):
    if hour_min is None:
        return None
    s = hour_min.split(":")
    if len(s) < 3:
        return (int(s[0]) * 60) + (int(s[1]))
    return (int(s[0]) * 60) + (int(s[1])) + int(s[2])/60

@functions.udf
def convert_actual_time(date, schedule_time, actual_time):
    if schedule_time == None or actual_time == None:
        return None
    # Day of trip
    arrival_date = schedule_time.split(" ")[0]
    arrival_min = to_sec_from_midnight(schedule_time.split(" ")[1])
    actual_arrival_date = actual_time.split(" ")[0]
    actual_arrival_min = to_sec_from_midnight(actual_time.split(" ")[1])
    
    # Add 24 hours to the actual arrival to get the correct differance
    if (arrival_date != actual_arrival_date):
        actual_arrival_min += 60*24
    return truncate(actual_arrival_min - arrival_min, 2)

dist_df = dist_df.withColumn("arrival_diff", convert_actual_time(dist_df.date, 
                                                                 dist_df.arrival_schedule, 
                                                                 dist_df.arrival_actual))

dist_df = dist_df.select('date', 'day_of_week', 'trip_id', "arrival_diff", line_day_stop_count)

dist_df = dist_df.filter(dist_df.stop_count>1)

sat_dist = dist_df.filter(dist_df.day_of_week == 7)
sun_dist = dist_df.filter(dist_df.day_of_week == 1)
week_dist = dist_df.filter((dist_df.day_of_week > 1) & (dist_df.day_of_week < 7))

sat_dist = sat_dist.groupBy("trip_id").agg(avg("arrival_diff").alias('delay_mean'), \
                                           stddev("arrival_diff").alias('delay_sd'))
sun_dist = sun_dist.groupBy("trip_id").agg(avg("arrival_diff").alias('delay_mean'), \
                                           stddev("arrival_diff").alias('delay_sd'))
week_dist = week_dist.groupBy("trip_id").agg(avg("arrival_diff").alias('delay_mean'), \
                                             stddev("arrival_diff").alias('delay_sd'))

In [33]:
# Loads the pandas df for further processing (the conversion from spark to pandas is slow so 
# we avoid the conversion every time we run)
# We can simply change LOAD_FROM_MEM to False to redo the conversion and dump the result to disk
LOAD_FROM_MEM = True
begin = time.time()
if not LOAD_FROM_MEM:
    sat_dist_pd = sat_dist.toPandas()
    sun_dist_pd = sun_dist.toPandas()
    week_dist_pd = week_dist.toPandas()
    pickle.dump(sat_dist_pd, open("./graph_saved/sat_dist_pd.pickle", "wb"))
    pickle.dump(sun_dist_pd, open("./graph_saved/sun_dist_pd.pickle", "wb"))
    pickle.dump(week_dist_pd, open("./graph_saved/week_dist_pd.pickle", "wb"))
else:
    sat_dist_pd = pickle.load(open("./graph_saved/sat_dist_pd.pickle", "rb"))
    sun_dist_pd = pickle.load(open("./graph_saved/sun_dist_pd.pickle", "rb"))
    week_dist_pd = pickle.load(open("./graph_saved/week_dist_pd.pickle", "rb"))
print("Time taken:", time.time()-begin)

Time taken: 0.12800097465515137


## ll. Modelling the Public Transport Network

Now that we obtained the various schedules, we model the Zürich area stops into a network. We create a separate network for each schedule (i.e. saturdays, sundays and week days). In order to do so, for each schedule we first create a dataframe where the rows represent two stops connected by a given trip and subsequently create a graph in networkx from this data.

It is worth noting that we initially started working with Spark as the size of the data was signficant. However, following the obtention of the schedule, as the datasets now represented a single day, it allowed us to work locally with Pandas which considerably reduced running time of operations in general.

In [34]:
# cast the schedule arrival/departure times (represented as minutes from midnight) as ints
saturday_pd = saturday_pd.astype({"arrival_time_schedule": int, "departure_time_schedule": int})
weekday_pd = weekday_pd.astype({"arrival_time_schedule": int, "departure_time_schedule": int})
sunday_pd = sunday_pd.astype({"arrival_time_schedule": int, "departure_time_schedule": int})

In [35]:
# order each schedule by trip id, arrival and departure schedule times
saturday_pd = saturday_pd.sort_values(by=['trip_id', 'arrival_time_schedule', 'departure_time_schedule'])

sunday_pd = sunday_pd.sort_values(by=['trip_id', 'arrival_time_schedule', 'departure_time_schedule'])

weekday_pd = weekday_pd.sort_values(by=['trip_id', 'arrival_time_schedule', 'departure_time_schedule'])

Having the schedule sorted on trip id, schedule arrival and departure times, we now have a dataframe which illustrates each trip in a single day with the list of stops it goes through from start to finish in sequential order.

In [36]:
sunday_pd.head(6)

,trip_id,stop_name,arrival_time_schedule,departure_time_schedule,stop_count
92841,85:11:1507:001,Zürich HB,390,399,3
92842,85:11:1507:001,Zürich Oerlikon,405,406,3
92843,85:11:1507:001,Zürich Flughafen,411,413,3
65387,85:11:1509:001,Zürich HB,450,459,3
65386,85:11:1509:001,Zürich Oerlikon,465,466,3
65385,85:11:1509:001,Zürich Flughafen,471,473,3


To obtain the edges of our graph, we concatenate the dataframes above with a shifted version of itself (along the columns). As the stops are ordered sequentially, this results in the desired dataframe where the rows contain from and to stops for each trip which effectively represent edges in our graph.

In [37]:
# create empty row for the shift
dummy_row = pd.DataFrame([[0] * len(saturday_pd.columns)], columns=list(saturday_pd.columns.values))

In [38]:
saturday_left_pd  = dummy_row.append(saturday_pd)
saturday_right_pd = saturday_pd.append(dummy_row)

sunday_left_pd  = dummy_row.append(sunday_pd)
sunday_right_pd = sunday_pd.append(dummy_row)

weekday_left_pd  = dummy_row.append(weekday_pd)
weekday_right_pd = weekday_pd.append(dummy_row)

In [39]:
# reset index for the join
saturday_right_pd = saturday_right_pd.reset_index()
saturday_left_pd = saturday_left_pd.reset_index()

sunday_right_pd = sunday_right_pd.reset_index()
sunday_left_pd = sunday_left_pd.reset_index()

weekday_right_pd = weekday_right_pd.reset_index()
weekday_left_pd = weekday_left_pd.reset_index()

In [40]:
saturday_right_pd = saturday_right_pd.drop(['index'], axis=1)
saturday_left_pd = saturday_left_pd.drop(['index'], axis=1)

sunday_right_pd = sunday_right_pd.drop(['index'], axis=1)
sunday_left_pd = sunday_left_pd.drop(['index'], axis=1)

weekday_right_pd = weekday_right_pd.drop(['index'], axis=1)
weekday_left_pd = weekday_left_pd.drop(['index'], axis=1)

In [41]:
sat_graph = saturday_left_pd.join(saturday_right_pd, rsuffix='_r')
week_graph = weekday_left_pd.join(weekday_right_pd, rsuffix='_r')
sun_graph = sunday_left_pd.join(sunday_right_pd, rsuffix='_r')

In [42]:
# remove rows where the trip id does not match (as the last and first stops of adjacent trips overlap)
sat_graph = sat_graph[sat_graph.trip_id == sat_graph.trip_id_r]
sun_graph = sun_graph[sun_graph.trip_id == sun_graph.trip_id_r]
week_graph = week_graph[week_graph.trip_id == week_graph.trip_id_r]

In [43]:
# select and subsequently rename desired columns
sat_graph = sat_graph[['trip_id', 'stop_name', 'arrival_time_schedule', 'departure_time_schedule',
                      'stop_name_r', 'arrival_time_schedule_r', 'departure_time_schedule_r']]
sun_graph = sun_graph[['trip_id', 'stop_name', 'arrival_time_schedule', 'departure_time_schedule',
                      'stop_name_r', 'arrival_time_schedule_r', 'departure_time_schedule_r']]
week_graph = week_graph[['trip_id', 'stop_name', 'arrival_time_schedule', 'departure_time_schedule',
                      'stop_name_r', 'arrival_time_schedule_r', 'departure_time_schedule_r']]

In [44]:
sat_graph.columns = ['trip_id',
                     'src_stop_name', 'src_arrival_time_schedule', 'src_departure_time_schedule',
                     'dst_stop_name', 'dst_arrival_time_schedule', 'dst_departure_time_schedule']
sun_graph.columns = ['trip_id',
                     'src_stop_name', 'src_arrival_time_schedule', 'src_departure_time_schedule',
                     'dst_stop_name', 'dst_arrival_time_schedule', 'dst_departure_time_schedule']
week_graph.columns = ['trip_id',
                     'src_stop_name', 'src_arrival_time_schedule', 'src_departure_time_schedule',
                     'dst_stop_name', 'dst_arrival_time_schedule', 'dst_departure_time_schedule']

For the edge weights of our graph, we compute these by calculating the difference between the arrival time of the destination stop and the arrival time of the source stop. Note that we chose to compute the difference only based on arrival times as this includes the waiting time at a stop into the length of a journey. If the source departure time was used instead, this would result in a lower bound on the actual journey time.

In [45]:
# calculate travel time
sat_graph['travel_time'] = sat_graph.dst_arrival_time_schedule - sat_graph.src_arrival_time_schedule
sun_graph['travel_time'] = sun_graph.dst_arrival_time_schedule - sun_graph.src_arrival_time_schedule
week_graph['travel_time'] = week_graph.dst_arrival_time_schedule - week_graph.src_arrival_time_schedule

In addition to the network that can be generated from the possible itineraries offered by the Swiss public transport, we need to enrich this with the possibility of walking between two stations. We create a dataframe containing pairs of stations as rows where these are at most 400 meters apart and compute the time it takes to walk between the two assuming an average walking speed of 1.4 m/s.

In [46]:
product = pd.merge(zurich_area_stops.assign(foo=1), zurich_area_stops.assign(foo=1), on='foo')
product['distance'] = np.vectorize(haversine)(product.long_x, product.lat_x, product.long_y, product.lat_y)
product = product[product.stop_x != product.stop_y]
product = product[product.distance < 0.4]
walking_df = product[['stop_x', 'stop_y', 'distance']].reset_index().drop(['index'], axis=1)
walking_df['walk_time'] = round((walking_df.distance * 1000) / (1.4 * 60))

Finally, we create the saturday, sunday and week day graphs where vertices are stops and edges respresent the fact that either at least one trip connects the two stops directly or that these stops are not far apart and can be reached by foot.

In [47]:
import networkx as nx
from networkx import NetworkXNoPath
from random import randint
import time

In [48]:
# The graphs from networkx take an iterator as input
def edges_iter_other_df(df, walking_df):
    edges_dict = {}
    for index in range(len(df)):
        try:
            present = edges_dict[(df.iloc[index].src_stop_name, df.iloc[index].dst_stop_name)]
            # weight
            present[0].append(df.iloc[index].travel_time)
            # trip_id
            present[1].append(df.iloc[index].trip_id)
        except:
            edges_dict[(df.iloc[index].src_stop_name, 
                        df.iloc[index].dst_stop_name)] = ([int(df.iloc[index].travel_time)], [df.iloc[index].trip_id])
    
    
    for index in range(len(walking_df)):
        if not (walking_df.iloc[index].stop_x, walking_df.iloc[index].stop_y) in edges_dict.keys():
            edges_dict[(walking_df.iloc[index].stop_x, 
                        walking_df.iloc[index].stop_y)] = ([5+int(walking_df.iloc[index].distance)*16],[-1])
            
    for key in edges_dict.keys():
        yield (key[0], key[1], {'weight': min(edges_dict[key][0]), 'trip_id': tuple(edges_dict[key][1])})

In [49]:
# Build the graph
import pickle
LOAD_FROM_MEM = True
begin = time.time()
if not LOAD_FROM_MEM:
    G = nx.DiGraph()
    G.add_edges_from(edges_iter_other_df(sun_graph, product))
    pickle.dump(G, open("./graph_saved/Gsun.pickle", "wb"))
    print("Time taken:", time.time()-begin)
    G = nx.DiGraph()
    G.add_edges_from(edges_iter_other_df(sat_graph, product))
    pickle.dump(G, open("./graph_saved/Gsat.pickle", "wb"))
    print("Time taken:", time.time()-begin)
    G = nx.DiGraph()
    G.add_edges_from(edges_iter_other_df(week_graph, product))
    pickle.dump(G, open("./graph_saved/Gweek.pickle", "wb"))
print("Time taken:", time.time()-begin)

Time taken: 0.00016736984252929688


## lll. Creating the journey planner algorithm

Once we have all the edges, we build a graph out of it. We also include walking edges which we defined as being edges between 2 stations with a distance of less than 0.4 km. Walking edges have a special attribute in the graph.

Then we have a randomized algorithm for computing weighted shortest paths. It uses dijkstra as a starting point and then randomly removes edges from previous paths to compute other potential paths. All theses paths are then spit into subpaths which involve no walking. This is easily done by looking at the weight of the edges of the path in the graph since it has a special attribute if it is a walking edge.

We then try to find an actual trip for each sub-paths. This is done by filtering out all the rows in the original data which do not contain stops on the path. For each trip_id in the resulting df, we also filter out rows that aren't the first/last stop for a particular trip_id. Moreover, we also filter out trains/bus/tram that leave before the departure time and trains/bus/tram that leave much later (3h) than the departure time. This leaves us with a dataframe of all the potential trip_id's that could be taken. We then use a greedy algorithm. We order the rows in the df by stop sequence. Stop sequence is 0 for the first station in the trip_id and is largest for the last station in the trip_id. 

We then sequentially go through all the rows and have 2 different heuristics for choosing which trip_id to take. Either we take the train that bring us the furthest on the path, or we take trains that leave as early as possible. Both these approches are used and multiple trips are returned to the user. We also compute the probability of making the trip and if it happens to be too low, we increase the transit time and rerun.

SUMMARY:

- build graph from edges df
- compute multiple paths   
    - segment each of them (separate walking from train/bus/tram)
        - for each segmented path, find a non-walking planning
    - put all of this together by talking walking time into account
    - if probability is too low, increase transit time and rerun

In [50]:
# Segment out the walking from the train/tram/bus
def get_potential_stations_in_shortest_path(src, dest, day, random_tries=10):
    # Load the correct graph
    graph_name = ''
    file_path = "./graph_saved/"
    if day == 6:
        graph_name = 'Gsun.pickle'
    elif day == 5:
        graph_name = 'Gsat.pickle'
    else:
        graph_name = 'Gweek.pickle'
    G = pickle.load(open(file_path + graph_name, "rb"))
    trip_id_lst = []
    
    # First, we try to find a single shortest path
    try:
        initial_length, initial_path = nx.single_source_dijkstra(G, src, dest, weight='weight')
        for i in range(len(initial_path)-1):
            s1 = initial_path[i]
            s2 = initial_path[i+1]
            trip_id = G[s1][s2]['trip_id']
            trip_id_lst.extend(trip_id)
            
    except NetworkXNoPath:
        # there is no path
        print("There is no path!")
        return None
    
    # Now, we increase randomly the weight of an edge and recompute the shortest path
    # This allows us to find sub optimal paths that might have higher probabilities of succeeding
    to_return = [(initial_path, list(set(trip_id_lst)))]
    for tries in range(random_tries):
        G = pickle.load(open(file_path + graph_name, "rb"))
        trip_id_lst = []
        first_index = randint(0, len(to_return)-1)
        index = randint(0, len(to_return[first_index])-2)
        s1 = to_return[first_index][0][index]
        s2 = to_return[first_index][0][index+1]
        old_weight = G[s1][s2]['weight']
        G[s1][s2]['weight'] = 24*60*10 # some high number

        length, path = nx.single_source_dijkstra(G, src, dest, weight='weight')
            
        if path not in [i[0] for i in to_return]:
            for index in range(len(path)-1):
                s11 = path[index]
                s21 = path[index+1]
                trip_id = G[s11][s21]['trip_id']
                trip_id_lst.extend(trip_id)

            to_return.append((path, list(set(trip_id_lst))))


    return to_return

For the stochastic aspect of the journey planner, we decided to include the historical data on a trip id basis. Additionally, we assumed that the delays for different trips were independent. Therefore, for a calculated journey, its probability is determined as follows: the product of each trip's probability to arrive in time for the connection (for the next trip in the journey) and multiply this result with 0.99. We decided to add the final multiplication by 0.99 as our journey planner displays the 99% confidence interval of arriving in a specified time window. Hence, direct trips will always have a probability of 99% to arrive in the displayed time window.
In order to calculate the probabilites of being able to make the suggested changes in a journey and the aforementioned time window of 99% confidence, we verified the distribution of delays for a number of trips. These are plotted below. From the distributions observed, we decided that modelling the delays of a trip as a Gaussian distribution was appropriate. Each trip's distribution would have its empirical mean and variance as its parameters.

In [51]:
# Trip id's are selected from the zurich_df
z_trips = ["85:78:12523:001","85:11:20457:001", "85:3849:87909-02009-1", "85:3849:88103-02009-1"]

begin = time.time()
LOAD_FROM_MEM = True
if not LOAD_FROM_MEM:
    # Filter to keep only the wanted trip id's in the dataframe
    z_trips_pd = dist_df.filter(dist_df.trip_id.isin(z_trips)).toPandas()
    pickle.dump(z_trips_pd, open("./graph_saved/histogram_data.pickle", "wb"))
else:
    z_trips_pd = pickle.load(open("./graph_saved/histogram_data.pickle", "rb"))

print("Time taken:", time.time()-begin)

# Load or compute the meta data for each distinct trip_id from the zurich_df datafame
import time
begin = time.time()
LOAD_FROM_MEM = True
if not LOAD_FROM_MEM:
    meta_data = zurich_df.dropDuplicates(["trip_id"])
    meta_data = meta_data.toPandas()
    pickle.dump(meta_data, open("./graph_saved/distinct_ids.pickle", "wb"))
else:
    meta_data = pickle.load(open("./graph_saved/distinct_ids.pickle", "rb"))

print("Time taken:", time.time()-begin)

Time taken: 0.0059452056884765625
Time taken: 0.21390986442565918


In [52]:
# Clean the data to be plotted

first_train = z_trips_pd[z_trips_pd.trip_id == z_trips[0]].dropna()
second_train = z_trips_pd[z_trips_pd.trip_id == z_trips[1]].dropna()
third_train = z_trips_pd[z_trips_pd.trip_id == z_trips[2]].dropna()
fourth_train = z_trips_pd[z_trips_pd.trip_id == z_trips[3]].dropna()

first_train = first_train.astype({"arrival_diff": float})
second_train = second_train.astype({"arrival_diff": float})
third_train = third_train.astype({"arrival_diff": float})
fourth_train = fourth_train.astype({"arrival_diff": float})

In [53]:
import numpy as np
import scipy.special
import math

from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.io import push_notebook, show, output_notebook


output_notebook()

def make_plot(title, hist, edges, x, pdf, cdf):
    p = figure(title=title, tools='', background_fill_color="#fafafa")
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
           fill_color="navy", line_color="white", alpha=0.5)
    p.line(x, pdf, line_color="#ff8888", line_width=4, alpha=0.7, legend="PDF")
    p.line(x, cdf, line_color="orange", line_width=2, alpha=0.7, legend="CDF")

    p.y_range.start = 0
    p.legend.location = "top_left"
    p.legend.background_fill_color = "#fefefe"
    p.xaxis.axis_label = 'x'
    p.yaxis.axis_label = 'Pr(x)'
    p.grid.grid_line_color="white"
    return p


# Train Nr 1

measured = first_train.arrival_diff.values
hist, edges = np.histogram(measured, density=True, bins=50)

mu, sigma = first_train.loc[:, "arrival_diff"].mean(), first_train.loc[:, "arrival_diff"].std()

x = np.linspace(math.floor(measured.min()), math.ceil(measured.max()), 1000)
pdf = 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2))
cdf = (1+scipy.special.erf((x-mu)/np.sqrt(2*sigma**2)))/2

transportation_id = "85:78:12523:001"
t_type = meta_data[meta_data["trip_id"] == transportation_id].iloc[0, 5]
if t_type == "Zug" or t_type == None:
    t_type = "Train"

p1 = make_plot("Train Id: {}, {} (μ={}, σ={})".format(first_train.iloc[0, 2], t_type, round(mu, 2), round(sigma, 2)), 
               hist, edges, x, pdf, cdf)


# Train Nr 2

measured = second_train.arrival_diff.values
hist, edges = np.histogram(measured, density=True, bins=50)

mu, sigma = second_train.loc[:, "arrival_diff"].mean(), second_train.loc[:, "arrival_diff"].std()

x = np.linspace(math.floor(measured.min()), math.ceil(measured.max()), 1000)
pdf = 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2))
cdf = (1+scipy.special.erf((x-mu)/np.sqrt(2*sigma**2)))/2

transportation_id = "85:11:20457:001"
t_type = meta_data[meta_data["trip_id"] == transportation_id].iloc[0, 5]
if t_type == "Zug" or t_type == None:
    t_type = "Train"

p2 = make_plot("Train Id: {}, {} (μ={}, σ={})".format(second_train.iloc[0, 2], t_type, round(mu, 2), round(sigma, 2)), 
               hist, edges, x, pdf, cdf)


# Train Nr 3

measured = third_train.arrival_diff.values
hist, edges = np.histogram(measured, density=True, bins=50)

mu, sigma = third_train.loc[:, "arrival_diff"].mean(), third_train.loc[:, "arrival_diff"].std()

x = np.linspace(math.floor(measured.min()), math.ceil(measured.max()), 1000)
pdf = 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2))
cdf = (1+scipy.special.erf((x-mu)/np.sqrt(2*sigma**2)))/2

transportation_id = "85:3849:87909-02009-1"
t_type = meta_data[meta_data["trip_id"] == transportation_id].iloc[0, 5]
if t_type == "Zug" or t_type == None:
    t_type = "Train"
    
p3 = make_plot("Train Id: {}, {} (μ={}, σ={})".format(third_train.iloc[0, 2], t_type, round(mu, 2), round(sigma, 2)), 
               hist, edges, x, pdf, cdf)


# Train Nr 4

measured = fourth_train.arrival_diff.values
hist, edges = np.histogram(measured, density=True, bins=50)

mu, sigma = fourth_train.loc[:, "arrival_diff"].mean(), fourth_train.loc[:, "arrival_diff"].std()

x = np.linspace(math.floor(measured.min()), math.ceil(measured.max()), 1000)
pdf = 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2))
cdf = (1+scipy.special.erf((x-mu)/np.sqrt(2*sigma**2)))/2

transportation_id = "85:3849:88103-02009-1"
t_type = meta_data[meta_data["trip_id"] == transportation_id].iloc[0, 5]
if t_type == "Zug" or t_type == None:
    t_type = "Train"

p4 = make_plot("Train Id: {}, {} (μ={}, σ={})".format(fourth_train.iloc[0, 2], t_type, round(mu, 2), round(sigma, 2)), 
               hist, edges, x, pdf, cdf)


histograms = gridplot([p1,p2,p3,p4], ncols=2, plot_width=400, plot_height=400, toolbar_location=None)

handle = show(histograms, notebook_handle=True)

# Push to notebook
push_notebook(handle=handle)

Loading BokehJS ...

Above we have plotted four histograms of the difference between scheduled and actual arrival time for four trip ids. For each plot the probability desity function (PDF) and the cumulative distribution function (CDF) for a Gaussian distribution has been ploted with the mean and the standard deviation of the corresponding distribution of arrival time differences. The trip id and the transportation type can be found in the title for each histogram.

In [54]:
def calc_prob(sch_arr, sch_dep, mean_delay, std_delay, transit_time, walk_time=0):
    epsilon = sch_dep - sch_arr - (transit_time + walk_time)
    
    normalized = (epsilon - mean_delay)/std_delay
    
    return norm.cdf(normalized)

In [55]:
def get_probability_trip(trip_id_lst, walking_df, path_tim_format, schedule_df, historic_df, 
                         transit_time, confidence=0.99):
    prob = 1
    for i in range(len(path_tim_format)-1):
        stop_a = path_tim_format[i][-1]
        stop_b = path_tim_format[i+1][0]

        a_arr_time = schedule_df[(schedule_df.trip_id == trip_id_lst[i]) 
                               & (schedule_df.stop_name == path_tim_format[i][-1])].iloc[0,2]

        b_dep_time = schedule_df[(schedule_df.trip_id == trip_id_lst[i+1]) 
                               & (schedule_df.stop_name == path_tim_format[i+1][0])].iloc[0,3]

        mean_delay = historic_df[historic_df.trip_id == trip_id_lst[i]].iloc[0,1]
        std_delay = historic_df[historic_df.trip_id == trip_id_lst[i]].iloc[0,2]   

        if stop_a == stop_b:
            prob *= calc_prob(a_arr_time, b_dep_time, mean_delay, std_delay, transit_time)
        else:
            walking_time = walking_df[(walking_df.stop_x == stop_a) & (walking_df.stop_y == stop_b)].iloc[0,3]
            prob *= calc_prob(a_arr_time, b_dep_time, mean_delay, std_delay, transit_time, walk_time=walking_time)
    
    mean_delay = historic_df[historic_df.trip_id == trip_id_lst[-1]].iloc[0,1]
    std_delay = historic_df[historic_df.trip_id == trip_id_lst[-1]].iloc[0,2]
    arr_time = schedule_df[(schedule_df.trip_id == trip_id_lst[-1]) 
                           & (schedule_df.stop_name == path_tim_format[-1][-1])].iloc[0,2]
    
    arrival_stat = [arr_time + mean_delay, norm.ppf(confidence) * std_delay]
    prob *= confidence
    
    return prob, arrival_stat

In [56]:
# Compute the route based on the filtered df (see explanation above)
def find_railed_path(path, end, df, transit_time, debug=False, prefer_direct=True):
    index = 0
    itinerary = []
    original_path = path.copy()
    best_trip_id = None
    validated_paths = []
    
    # not really needed, only for debugging
    best_arrival_time = 24*60*10
    best_station_stop_name = None
    debug_best_stop_index = -1
    
    best_station_stop_index = -1
    offset = 0
    time = 0
    if debug:
        for i in range(4):
            print(20*'-')
        print(path)
        for i in range(4):
            print(20*'-')
    while path != []:
        if debug:
            print(index, len(df), best_trip_id, best_station_stop_index, path)

        try:
            if df.iloc[index].stop_name == path[0]:
                if debug:
                    print('INDSIDE IF')
                start_stop = df.iloc[index]
                end_stop = df.iloc[index+1]

                start_trip_id = start_stop.trip_id
                stop_trip_id = end_stop.trip_id
                assert(start_trip_id == stop_trip_id)
                if prefer_direct:
                    extra_cond_and = True
                    extra_cond_or = False
                else:
                    extra_cond_and = end_stop.arrival_time_schedule < best_arrival_time
                    extra_cond_or = end_stop.stop_index >= best_station_stop_index
                if extra_cond_and and (end_stop.stop_index > best_station_stop_index or extra_cond_or) and start_stop.departure_time_schedule >= time:
                    best_trip_id = start_trip_id
                    best_station_stop_index = end_stop.stop_index
                    debug_best_stop_index = end_stop.stop_index
                    best_arrival_time = end_stop.arrival_time_schedule
                    best_station_stop_name = end_stop.stop_name
                    
                if end_stop.stop_name == original_path[-1] and start_stop.departure_time_schedule >= time:
                    best_trip_id = start_trip_id
                    best_station_stop_index = end_stop.stop_index
                    debug_best_stop_index = end_stop.stop_index
                    best_arrival_time = end_stop.arrival_time_schedule
                    best_station_stop_name = end_stop.stop_name
                    itinerary.append(best_trip_id)
                    validated_paths.append(path[:best_station_stop_index-offset+1])
                    return itinerary, best_station_stop_index, validated_paths

            elif df.iloc[index].stop_index == df.iloc[index-2].stop_index:
                if best_trip_id is not None:
                    time = best_arrival_time + transit_time
                    itinerary.append(best_trip_id)
                    validated_paths.append(path[:best_station_stop_index-offset+1])
                    path = path[best_station_stop_index-offset:]
                    offset = best_station_stop_index
                    best_trip_id = None
                    best_arrival_time = 24*60*10
        
                if best_station_stop_index == len(original_path)-1:
                    return itinerary, best_station_stop_index, validated_paths

                #########################################################
                if df.iloc[index].stop_name == path[0]:
                    if debug:
                        print('INDSIDE IF')
                    start_stop = df.iloc[index]
                    end_stop = df.iloc[index+1]

                    start_trip_id = start_stop.trip_id
                    stop_trip_id = end_stop.trip_id
                    assert(start_trip_id == stop_trip_id)
                    if prefer_direct:
                        extra_cond_and = True
                        extra_cond_or = False
                    else:
                        extra_cond_and = end_stop.arrival_time_schedule < best_arrival_time
                        extra_cond_or = end_stop.stop_index >= best_station_stop_index
                    if extra_cond_and and (end_stop.stop_index > best_station_stop_index or extra_cond_or) and start_stop.departure_time_schedule >= time:
                        best_trip_id = start_trip_id
                        best_station_stop_index = end_stop.stop_index
                        debug_best_stop_index = end_stop.stop_index
                        best_arrival_time = end_stop.arrival_time_schedule
                        best_station_stop_name = end_stop.stop_name
                ##########################################################
            index = index + 2
            
        except IndexError:
            if debug:
                print("The last stop we got to is:", best_station_stop_name, 
                      "and arrives at", best_arrival_time)
                print(original_path)
                print(df.head(100))
            if best_trip_id is not None:
                
                if best_station_stop_name == original_path[-1]:
                    itinerary.append(best_trip_id)
                    validated_paths.append(path)
                    return itinerary, best_station_stop_index, validated_paths

            return None, None, None

    return itinerary, best_station_stop_index, validated_paths

In [57]:
def get_stops_df(path, start_time, all_stops_df):
    stops = all_stops_df[all_stops_df.stop_name.isin(path)]
    stops = stops[(stops.departure_time_schedule >= start_time) & (stops.arrival_time_schedule < start_time + 180)]

    journeys = stops.groupby('trip_id').count().iloc[:,0:1]
    journeys.columns = ['stop_count']

    stops = stops[stops.trip_id.isin(journeys[journeys.stop_count != 1].index.values)]
    stops = stops.join(pd.DataFrame(index=path, data=list(range(len(path))), columns=['stop_index']), on='stop_name')
    stops['ranker'] = ((stops.arrival_time_schedule + stops.departure_time_schedule) / 2) + 0.001 * stops.stop_index
    
    

    g = stops.groupby('trip_id')
    stops['arrival_rank'] = g['ranker'].rank()
    stops['departure_rank'] = g['ranker'].rank(ascending=False)
    
    before_filter = stops.copy()
    
    stops = stops[(stops.arrival_rank == 1) | (stops.departure_rank == 1)]

    stops = stops.sort_values(by=['trip_id','stop_index', 'departure_time_schedule'])
    fwd_trips = []
    for s in stops.trip_id.drop_duplicates().values:
        trip = stops[stops.trip_id == s]
        if trip.iloc[0].arrival_time_schedule < trip.iloc[1].departure_time_schedule:
            fwd_trips.append(s)

    stops = stops[stops.trip_id.isin(fwd_trips)]

    order = stops.sort_values(['stop_index', 'departure_time_schedule']).trip_id.drop_duplicates().values
    stops['trip_id'] = pd.Categorical(stops['trip_id'], order)

    stops = stops.sort_values(['trip_id', 'departure_time_schedule']).set_index('trip_id').reset_index()

    stops = stops.drop(columns=['ranker'])
    
    return stops, before_filter

In [58]:
# This is the main function that is called by the client that displays the trips
def get_trips_information(src, dst, day_of_week, departure_time, probability_threshold, debug=False):
    '''Returns a list of dicts where each dict contains:    -arrival_time
                                                            -departure_time
                                                            -path (list of non walking paths)
                                                            -trip_ids (list of trip_ids)
                                                            -probability'''

    if day_of_week == 6:
        df = sunday_pd
        historic_df = sun_dist_pd
    elif day_of_week == 5:
        df = saturday_pd
        historic_df = sat_dist_pd
    else:
        df = weekday_pd
        historic_df = week_dist_pd
    
    original_transit_time = 2
    transit_time = 2
    
    if len(walking_df[(walking_df.stop_x == src) & (walking_df.stop_y == dst)]) == 1:
        walk_time = walking_df[(walking_df.stop_x == src) & (walking_df.stop_y == dst)].walk_time
        
        return {'arrival_time': int(walk_time) + departure_time, 'departure_time': departure_time, "path": [], 
                      "probability": 1.0, "trip_ids": [], 
                      "expected_arrival_time": float(departure_time + walk_time), "confidence_interval": 0.0, 
                     "validated_path": []}

    all_possible_paths = get_potential_stations_in_shortest_path(src, dst, day_of_week)
    if all_possible_paths is None:
        return None
    lst_of_trip_dict = []
    # loop over all possible found physical paths
    for p in all_possible_paths:
        old_single_dict = {}
        # loop over prefer direct trains or fast trains
        for boolean_direct in (True, False):
            try:
                single_dict = get_single_trip_information(get_all_sub_paths(p[0], day_of_week), departure_time, df, historic_df, 
                                                          transit_time, debug=debug, prefer_direct=boolean_direct)

                if single_dict != {}:
                    if old_single_dict != single_dict:
                        repeat_index = 0
                        # loop with higher transit time to increase the probability of arriving on time
                        while repeat_index < 3 and single_dict["probability"] < probability_threshold:
                            transit_time = transit_time + 5 # random number
                            single_dict = get_single_trip_information(get_all_sub_paths(p[0], day_of_week), 
                               departure_time, df, historic_df, transit_time, debug=debug, prefer_direct=boolean_direct)
                            if single_dict == {}:
                                single_dict = {"probability": 0.0}
                            repeat_index += 1    
                        transit_time = original_transit_time
                        if single_dict["probability"] >= probability_threshold:
                            if single_dict not in lst_of_trip_dict:
                                lst_of_trip_dict.append(single_dict)
                    old_single_dict = single_dict
            except:
                assert(1 == 1) # let it be
        
    filtered_lst_of_trip_dict = []
    for i in lst_of_trip_dict:
        not_in_bool = True
        probi = i["probability"]
        departure_timei = i["departure_time"]
        expected_arrival_timei = i["expected_arrival_time"]
        try:
            for j in filtered_lst_of_trip_dict:
                probj = j["probability"]
                departure_timej = j["departure_time"]
                expected_arrival_timej = j["expected_arrival_time"]
                if probi == probj and departure_timej == departure_timei and expected_arrival_timej == expected_arrival_timei:
                    not_in_bool = False
        except:
            assert(1 == 1) # iter over empty list at first
        if not_in_bool:
            filtered_lst_of_trip_dict.append(i)
    return filtered_lst_of_trip_dict

In [59]:
def get_all_sub_paths(path, day):
    # Segments out the paths that are separated by a walking distance
    graph_name = ''
    file_path = "./graph_saved/"
    if day == 6:
        graph_name = 'Gsun.pickle'
    elif day == 5:
        graph_name = 'Gsat.pickle'
    else:
        graph_name = 'Gweek.pickle'
    G = pickle.load(open(file_path + graph_name, "rb"))
    
    walking_start_station_lst = []
    for path_index in range(len(path)-1):
        s1 = path[path_index]
        s2 = path[path_index+1]
        if G[s1][s2]['trip_id'] == (-1,):
            walking_start_station_lst.append(s1)

    if walking_start_station_lst == []:
        paths = [path]
    else:
        sub_path = []
        index_walking = 0
        currently_railed = False
        paths = []
        for index, stop in enumerate(path):
            if stop == walking_start_station_lst[0]:
                if currently_railed:
                    sub_path.append(stop)
                    
                if sub_path != []:
                    paths.append(sub_path)
                sub_path = []
                walking_start_station_lst = walking_start_station_lst[1:]
                if walking_start_station_lst == []:
                    for i in path[index+1:]:
                        sub_path.append(i)
                    if sub_path != []:
                        paths.append(sub_path)
                    break
            else:
                sub_path.append(stop)
                currently_railed = True
    return paths

In [60]:
def get_single_trip_information(path, departure_time, all_stops_df, historic_df, transit_time, 
                                debug=False, prefer_direct=True):
    '''Returns a list of a dict containing:                 -arrival_time
                                                            -departure_time
                                                            -path (list of non walking paths)
                                                            -trip_ids (list of trip_ids)
                                                            -probability'''
    ready_to_leave_time = departure_time
    trip_id_lst = []
    total_validated_paths = []
    total_validated_paths_real = []
    walk_time = 0
    for index, p in enumerate(path):
        if len(p) != 1:
            df, df_before_filter = get_stops_df(p, ready_to_leave_time, all_stops_df)

            trip_lst_sub, end_stop_sub, validated_paths = find_railed_path(p, p[-1], df, transit_time,
                                                                           debug=debug, prefer_direct=prefer_direct)
            if trip_lst_sub is None:
                return {}
            
            total_validated_paths.extend(validated_paths)
            last_trip_id = trip_lst_sub[-1]
            tiny_df = df[df.trip_id == last_trip_id]
            tiny_df.sort_values(by=['arrival_time_schedule'])
            arrival_time = tiny_df.iloc[1].arrival_time_schedule

            if index != len(path)-1:
                walk_time = walking_df[(walking_df.stop_x == path[index][-1]) & (walking_df.stop_y == path[index+1][0])].walk_time


            ready_to_leave_time = arrival_time + int(walk_time) + transit_time

            trip_id_lst.append(trip_lst_sub)

            if index == 0:
                first_trip_id = trip_id_lst[0][0]
                tiny_df = df[df.trip_id == first_trip_id]
                tiny_df.sort_values(by=['departure_time_schedule'])
                departure_time = tiny_df.iloc[0].departure_time_schedule
                
            ################################
            # Get real paths taken (which might differ a bit from the ones found from the graph)
            for index_real, trip_id_real in enumerate(trip_lst_sub):
                stopped_at = validated_paths[index_real][-1]
                
                stopped_at_index = df[(df.trip_id == trip_id_real) & (df.stop_name == stopped_at)].iloc[0].arrival_rank
                
                filtered_sub_path = list(df_before_filter[(df_before_filter.trip_id == trip_id_real) & (df_before_filter.arrival_rank <= stopped_at_index)].stop_name.values)
                total_validated_paths_real.append(filtered_sub_path)
            
            ################################
            
        else:
            if index == 0:
                walk_time = walking_df[(walking_df.stop_x == path[index][-1]) & (walking_df.stop_y == path[index+1][0])].walk_time
                ready_to_leave_time = walk_time + departure_time
            elif index == len(path)-1:
                assert(int(ready_to_leave_time) - int(transit_time) == int(arrival_time) + int(walk_time))
                arrival_time = ready_to_leave_time - transit_time
            else:
                print("Attempting to walk more than 400m...")
                
            
                
    
                
    probability, arrival_stat = get_probability_trip([i for i in iterator_trip_id(trip_id_lst)], 
                                                     walking_df, total_validated_paths, 
                                                     all_stops_df, historic_df, 2)

    to_return_dict = {'arrival_time': arrival_time, 'departure_time': departure_time, #"path": path, 
                      "probability": probability, "trip_ids": [i for i in iterator_trip_id(trip_id_lst)], 
                      "expected_arrival_time": arrival_stat[0]+float(walk_time), "confidence_interval": arrival_stat[1], 
                     "validated_path": total_validated_paths_real}
    return to_return_dict

def iterator_trip_id(trip_id):
    for a in trip_id:
        for b in a:
            yield b

## IV. Visualization of Results

Now that we have a algorithm for calculating the shortes from a starting point to a destination we can start querying it and visualize the result. Each query to the algorithm will contain 5 parameters; *Source*, *Destionation*, *Day*, *Time* and *Probability*. The first two parameters, `src` and `dest` needs to be strings and be contained in the stations which are in a 10km radius of Zürich HB station. The third parameter, `day_of_week`, will need to be given as a string, Monday to Sunday. The fourth parameter,`user_time`, will need to be on the format "HH:MM" to be accepted and the the last parameter, `probability`, can be a Integer, Float or a String to be accepted. If one or multiple of the last three parameters are missing from the query, the algorithm will use the current day and time and a probability of 80% to calculate the paths.

Once all of the input parameters have been sanitized, we will fetch the paths from. These are returned in a List containing  a dictionaries for each possible path. Each dictionary will contain the following fields:
- arrival_time: Arrival time of the transportation vehicle.
- departure_time: Drrival time of the transportation vehicle.
- path: A list of paths where each path doesn't require any walking between stations. This means that there can be multiple means of transportation in one path.
- probability: The probability of getting to the destionation.
- trip_ids: A list of trip ids of the means of transportation.
- expected_arrival_time: The expected arrival time at the destination.
- confidence_interval: The confidence interval of the expected arrival time.
- validated_path: A list of paths where each path is corresponding to a trip_id, unlike paths in the paths field. A list of stops in the validated_path list will from now on be called a sub path.

To create the map we will iterate through each path returned by the algorightm and add each line of the different sub paths to the figure alongside a HoverTool which makes it possible to se the start and stop of this sub path. Each line will also be added to a Legend for the whole path as to make it possible to mute and unmute each path. While adding the line we will store the stations that our paths go through as these will aslo be added to the map as white dots. One can add all stations to the map by setting the `plot_all_stations` variable to true. The final map we contain the relevant stations for the paths (or all by the users choice) and each path found by the algorithm. The map will always start in such a position such that the whole path is visible without moving it or using the zoom tool.

In [61]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.tile_providers import get_provider, Vendors
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.palettes import Category20, Plasma256, viridis
from pyproj import Proj, transform
import datetime
import calendar
import re
import math

# This cell is mostly helper functions
# Function for computing time from a integer that represents minutes from midnight
# Input should be a positive integer and output will be time on the format "HH:MM"
def min_from_midnight_to_time(time):
    mins = time % 60
    hours = time // 60
    if mins < 10:
        mins = "0" + str(mins)
    if hours < 10:
        hours = "0" + str(hours)
    return str(hours) + ":" + str(mins)

# Function for computing the minutes from midnight from a string on the format "HH:MM"
# Output will be an integer
def min_from_midnight(time_string):
    values = time_string.split(":")
    return int(values[0]) * 60 + int(values[1])

# Function for adding a line to the figure containing the map. Input:
# - index: integer
# - fig: Figure Object
# - lats, longs: List containg mercator latitude and longitude values for the line to plot
# - color: string
# - path_info: dictionary containing all information about the current path
# - from_station: string, starting point of the line
# - to_station: string, end of the line
# - walking: boolen, decides if the line is a walking path or not
# - t_type: string, type of transportation
def add_line_and_hovertool(index, fig, lats, longs, color, path_info,
                           from_station, to_station, walking=False, t_type=None):
    
    # If the walking is true, plot a dotted line instead. 1 is the length of the dot 
    # and 8 is the separation between dots
    line_style = "solid"
    if walking:
        line_style = [1, 8]
    
    # Add line. Legend will contain the number of the path (out of all paths found), departure time, arrival time with
    # the confidence intervall for the arrival and the probability of the path
    lines = fig.line(x=lats, y=longs, color=color, line_dash=line_style, muted_color="gray", line_join="round", 
                     legend=str(index+1) + ": " + min_from_midnight_to_time(path_info["departure_time"]) + " - " + 
                     min_from_midnight_to_time(int(path_info["expected_arrival_time"])) + "+-" + 
                     min_from_midnight_to_time(int(math.ceil(path_info["confidence_interval"]))) + ", Prob: " + 
                     str(round(path_info["probability"] * 100, 2)) + "%", 
                     alpha=1, muted_alpha=0.2, line_width=5, line_cap="round")
    
    # Mute all lines exept the lines from the first path in the dictionary
    if index != 0:
        lines.muted = True
    
    # Different HoverTool for walking path and travel path
    if walking:
        fig.add_tools(HoverTool(tooltips="Walk from {} to {}".format(from_station, to_station), 
                                mode='mouse', renderers=[lines]))
    else:
        fig.add_tools(HoverTool(tooltips="Take {} from {} to {}".format(t_type, from_station, to_station), 
                                mode='mouse', renderers=[lines]))
        
# Returns the mercator coordinates for a given stop, input is a string
def get_mercator(stop):
    current_stop = zurich_area_stops[zurich_area_stops.stop == stop]
    return transform(Proj(init='epsg:4326'), Proj(init='epsg:3857'), float(current_stop.iloc[0].long), 
                     float(current_stop.iloc[0].lat))

# Returns a list of longitudes and a list of latitudes, input is a list of stations
def get_line_coordinates(stations):
    lats, longs = [], []
    for stop in stations:
        coor = get_mercator(stop)
        lats.append(coor[0])
        longs.append(coor[1])
    return lats, longs

In [62]:
# Plot every station within a 10 km radius of Zürich HB
# If false, only plot the stations that are in the paths
plot_all_stations = False

# Metode for sanitization of possible user provided data
# Input:
# src: String
# dest: String
# day_of_week: String, will be converted to a integer, so needs to be valid
# user_time: String, should be on the format "HH:MM" as it will be converted to a intger
# probability: Integer/Float/String
def search_trip(src, dest, day_of_week=None, user_time=None, probability=None):
    current_time = datetime.datetime.now()
    
    # Sanitize user data
    # src and dest, return if they are not in the list of stops
    if zurich_area_stops[(zurich_area_stops.stop == src) | 
                         (zurich_area_stops.stop == dest)].shape[0] != 2:
        print("Please input a valid departure and/or arrival station")
        return
    
    # Day of the week
    # Return if the given day isn't right, else parse to a integer (0 = Monday, 6 = Sunday)
    # If no data is given, use todays day
    # print(src, dest, day_of_week, user_time, probability)
    if day_of_week != None:
        if day_of_week in [i for i in calendar.day_name]:
            day_of_week = time.strptime(day_of_week, "%A").tm_wday
        else:
            print("Please input one of these valid days: {}".format([i for i in calendar.day_name]))
            return
    else:
        day_of_week = current_time.weekday()
    
    # Time
    # Check if the given time is on the right format ("HH:MM"). If not return, else parse it to minute after midnight
    # If no data is given use the current time
    time_regex = r"^([0-9]{2}):([0-9]{2}$)"
    if user_time != None:
        if re.search(time_regex, user_time) == None:
            print("Please input a valid time in the fromat: HH:MM")
            return
        else: 
            user_time_list = re.search(time_regex, user_time).string.split(":")
            if int(user_time_list[0]) > 24 or int(user_time_list[1]) > 59:
                print("Please input a valid time in the fromat: HH:MM")
                return
            user_time = int(min_from_midnight(re.search(time_regex, user_time).string))
    else:
        user_time = int(min_from_midnight(current_time.strftime("%H:%M")))
    
    # Probability
    # Check if the given probability is on the right format, can be string or float 
    # (string: "XXX.XXX...", float: XXX.XXX...)
    # Will divide given probability on 100
    # If no data is given use 0.8
    probability_regex = r"^([0-9]{1,3})(\.?)[0-9]*$"
    if probability != None:
        probability = str(probability)
        if re.search(probability_regex, probability) != None:
            probability = float(re.search(probability_regex, probability).string)
            if probability > 100.0:
                print("{} is not valid, please return a valid probability on the form XXX.XX".format(probability))
                return
            probability = probability / 100
        else:
            print("{} is not valid, please return a valid probability on the form XXX.XX".format(probability))
            return
    else:
        probability = 0.8
        
    # print(src, dest, day_of_week, user_time, probability)
    
    # Make the map
    make_map(src, dest, day_of_week, user_time, probability)
    
# Metode for making the map
# Input:
# src: String
# dest: String
# day_of_week: int
# user_time: int
# probability: float
def make_map(src, dest, day_of_week, current_time, probability):

    output_notebook()
    
    # Get the paths found by the algorithm
    paths = get_trips_information(src, dest, day_of_week, current_time, probability, debug=False)
    print("{} paths found".format(len(paths)))

    # Create a ColumnStore for the stations that will be plotted, it will contain the latitude and longitude of
    # the stations, the station names and  the circle sizes to be ploted
    latitude, longitude, stations, circle_size = [], [], [], []
    trains_info_dict = dict(latitude=latitude, longitude=longitude, station=stations, circle_size=circle_size)
    source_stations = ColumnDataSource(data=trains_info_dict)
    
    # Add src and dest stations 
    stations = [src, dest]

    # Get x and y range of src and dest so the map will start by showing the paths that are plotted
    src_coor = get_mercator(src)
    dest_coor = get_mercator(dest)

    x_range = (min(src_coor[0], dest_coor[0]) - 2000, max(src_coor[0], dest_coor[0]) + 2000)
    y_range = (min(src_coor[1], dest_coor[1]) - 2000, max(src_coor[1], dest_coor[1]) + 2000)
    
    # Create the map
    fig = figure(tools=['pan', 'wheel_zoom'], x_range=x_range, y_range=y_range, 
                 plot_width=800, plot_height=600)
    tile_provider = get_provider(Vendors.CARTODBPOSITRON)
    fig.add_tile(tile_provider)
        
    # Iterate over each path, that will be plotted
    for index, path_info in enumerate(paths):
        # List for printing the path in the end
        to_print = ["Path nr. {}".format(index + 1)]
        
        # If the validated_path is empty one will walk from src to dest
        if not path_info["validated_path"]:
            # Get the mercator coordinates for the line, add the line and add text to be printed
            lat, long = get_line_coordinates([src, dest])
            add_line_and_hovertool(index, fig, lat, long, "royalblue", path_info, src, dest, 
                                   walking=True)
            to_print.append("Walk from {}, to {}".format(src, dest))
        
        
        number_of_trips = len(path_info["validated_path"])
        
        # Variable used to find out if the end stop of a sub path is the same as the start stop of next sub path
        path_src = src
        # print("Lenght of path is {}".format(len(path_info["validated_path"])))
        
        #Iterate of the sub paths in a path. Get a color for each sub path from a dynamic list from viridis (based
        # on the number of sub paths in a path)
        for s_index, (sub_path, color) in enumerate(zip(path_info["validated_path"], 
                                                        viridis(len(path_info["validated_path"])))):
            
            # Add walking path if there is a need to switch transportation
            # Last stop of previous sub path or src != first station of the next sub path
            first_travel_stop = sub_path[0]
            if path_src != first_travel_stop:
                # Get the mercator coordinates for the line, add the line and add text to be printed
                walking_stations = [path_src, first_travel_stop]
                lat, long = get_line_coordinates(walking_stations)
                add_line_and_hovertool(index, fig, lat, long, "royalblue", path_info, path_src, first_travel_stop, 
                                       walking=True)
                to_print.append("Walk from {}, to {}".format(path_src, first_travel_stop))

            # Add transportation path
            lats = []
            longs = []
            
            # Get transportation type
            transportation_id = path_info["trip_ids"][s_index]
            t_type = meta_data[meta_data["trip_id"] == transportation_id].iloc[0, 5]
            if t_type == "Zug" or t_type == None:
                t_type = "Train"

            # Add stops in the sub_path to stations so they will be ploted
            for stop in sub_path:
                coor = get_mercator(stop)
                lats.append(coor[0])
                longs.append(coor[1])

                # Add station to stations to plot
                current_stop = zurich_area_stops[zurich_area_stops.stop == stop]
                if current_stop.iloc[0].stop not in stations:
                    stations.append(current_stop.iloc[0].stop)
                    
            # Add the current sub path to the map
            add_line_and_hovertool(index, fig, lats, longs, color, path_info, sub_path[0], sub_path[-1], t_type=t_type)
            to_print.append("Take {} from {}, to {}".format(t_type, sub_path[0], sub_path[-1]))
            
            # Add last walking path if it is needed to get to the destination
            if sub_path[-1] != dest and s_index == number_of_trips - 1:
                # Get the mercator coordinates for the line, add the line and add text to be printed
                walking_stations = [dest, sub_path[-1]]
                lats, longs = get_line_coordinates(walking_stations)
                add_line_and_hovertool(index, fig, lats, longs, "royalblue", path_info, sub_path[-1], dest, walking=True)
                to_print.append("Walk from {}, to {}".format(sub_path[-1], dest))
                
            # Update the src for the next sub path to the end of the current sub path
            path_src = sub_path[-1]
        
        # Print the path
        for s in to_print:
            print(s)
        print()
        
    # Add latitude, longitude and circle size for the given station in the paths
    for station in stations:
        coor = get_mercator(station)
        latitude.append(coor[0])
        longitude.append(coor[1])
        if station == src or station == dest:
            circle_size.append(12)
        else:
            circle_size.append(8)

    # Add circles and lines
    stops = fig.circle(x="latitude", y="longitude", size="circle_size", source=source_stations, alpha=1, 
                       line_width=2, line_color="black", color="white")
    
    # Add hovertools to stops
    fig.add_tools(HoverTool(renderers=[stops], tooltips=[("Index", "$index"), ("Station", "@station")], mode='mouse'))

    handle = show(fig, notebook_handle=True)

    # Code for ploting all stations if the variable is set to true
    if plot_all_stations:
        latitude, longitude, station, circle_size = [], [], [], []
        for index, row in zurich_area_stops.iterrows():
            long = row["long"]
            lat = row["lat"]
            stop = row["stop"]
            coor = transform(Proj(init='epsg:4326'), Proj(init='epsg:3857'), long, lat)
            latitude.append(coor[0])
            longitude.append(coor[1])
            station.append(stop)
            circle_size.append(8)

    # Update the ColumnStores with the data for stations and paths
    source_stations.data = dict(latitude=latitude, longitude=longitude, station=stations, circle_size=circle_size)

    # Add legend
    if (len(paths) != 0):
        fig.legend.location = "top_left"
        fig.legend.click_policy = "mute"

    # Push to notebook
    push_notebook(handle=handle)

Before the map is plotted the different paths and sub paths will be printed out. Here one can find information about how one shall get from one station to another and in what order.

When looking at the map below there are a few thing that are worth noticing:
- The start station and end station for the paths are the two circles which are sligtly bigger than the rest.
- The first path will always be the one that doesn't start as muted. Muted paths are shown in a transparent grey color.
- By interacting with the Legend in the top left corner, one can choose which of the paths that are going to be muted or unmuted.
- Moving the mouse over one of the stops that reperesent a station will trigger a pop-up box containing information about the station name.
- Moving the mouse over one of the paths will trigger a pop-up box containing information about that very sub path, dipslaying the means of transportation used and the start and destionation station.
- Sub paths in one path are displayed in colors from blue to green to yellow with sub paths earlier sub paths taking on a blue color and later a yellow one. 
- Sub paths that consists of walking from one station to another will be displayed as a dotted blue line.

In [63]:
################################################# VALIDATION #########################################################

Example where the sbb app prefered trip has a low probability.

In [64]:
search_trip(src='Urdorf Weihermatt', dest='Wallisellen', day_of_week='Saturday', user_time='22:40', probability=10)

Loading BokehJS ...

2 paths found
Path nr. 1
Take Train from Urdorf Weihermatt, to Zürich Hardbrücke
Take Train from Zürich Hardbrücke, to Zürich Oerlikon
Take Train from Zürich Oerlikon, to Wallisellen

Path nr. 2
Walk from Urdorf Weihermatt, to Urdorf, Uitikonerstrasse
Take Bus from Urdorf, Uitikonerstrasse, to Schlieren, Brunngasse
Walk from Schlieren, Brunngasse, to Schlieren
Take Train from Schlieren, to Zürich Hardbrücke
Take Train from Zürich Hardbrücke, to Zürich Oerlikon
Take Train from Zürich Oerlikon, to Wallisellen



By increasing the probability, the same path but talking other trains. We arrive later but the probability is 99%.

In [65]:
search_trip(src='Urdorf Weihermatt', dest='Wallisellen', day_of_week='Saturday', user_time='22:40', probability=80)

Loading BokehJS ...

2 paths found
Path nr. 1
Take Train from Urdorf Weihermatt, to Zürich Hardbrücke
Take Train from Zürich Hardbrücke, to Zürich Oerlikon
Take Train from Zürich Oerlikon, to Wallisellen

Path nr. 2
Walk from Urdorf Weihermatt, to Urdorf, Uitikonerstrasse
Take Bus from Urdorf, Uitikonerstrasse, to Schlieren, Brunngasse
Walk from Schlieren, Brunngasse, to Schlieren
Take Train from Schlieren, to Zürich Hardbrücke
Take Train from Zürich Hardbrücke, to Zürich Oerlikon
Take Train from Zürich Oerlikon, to Wallisellen



This is an example of a path that is 25 minutes faster than the sbb app but has a low probability of succeeding.

In [67]:
search_trip(src="Kloten, Freienberg", dest="Thalwil, In Reben", day_of_week="Tuesday", 
            user_time="20:20", probability=10.0)

Loading BokehJS ...

1 paths found
Path nr. 1
Take Bus from Kloten, Freienberg, to Zürich Flughafen, OPC
Walk from Zürich Flughafen, OPC, to Zürich Flughafen
Take Train from Zürich Flughafen, to Thalwil
Walk from Thalwil, to Thalwil, Zentrum
Take Bus from Thalwil, Zentrum, to Thalwil, Trotte
Walk from Thalwil, Trotte, to Thalwil, In Reben



Here is the same query as above, but with probability set to 80%

In [68]:
search_trip(src="Kloten, Freienberg", dest="Thalwil, In Reben", day_of_week="Tuesday", 
            user_time="20:20", probability=80.0)

Loading BokehJS ...

1 paths found
Path nr. 1
Take Bus from Kloten, Freienberg, to Zürich Flughafen, OPC
Walk from Zürich Flughafen, OPC, to Zürich Flughafen
Take Train from Zürich Flughafen, to Thalwil
Walk from Thalwil, to Thalwil, Zentrum
Take Bus from Thalwil, Zentrum, to Thalwil, Trotte
Walk from Thalwil, Trotte, to Thalwil, In Reben



# Limitations

Although we have now a working planner that integrates the probabilistic aspect of trips, our implementation has a number of limitations. Limitations that are worth taking into account are the following:
- The week day, saturday and sunday schedules were each defined from a sample of 1 day. As a result, if certain trips have failed on those days that were sampled, these would not be included (we verified that the number of such trips was insignificant). The decision to sample 1 day stemmed from the fact that a specific trip on a given day of the week could potentially boast a different trip id the following week. This created numerous issues when we attempted to recover the schedule from a larger sample of days.
- Following from the point about the trip ids being sometimes inconsistent, another limitation in our implementation is the fact that we define the probabilities for a given journey based on the historical punctuality of the trip ids that compose it. As a result, when calculating the journey, the historical data collected for a trip would potentially be a subset of the total data available for that given trip as, historically, it possibly relates to multiple trip ids.
- The fact that we modelled the public transport network as a graph for each different day type (saturday, sunday, week day) also carries certain flaws. The graph illustrates stops as vertices and two vertices are connected by an edge if a transport directly connects them. A problematic scenario is where two stops are connected directly, but there is only one transport that connects them and it operates once a day (such a night buses/trams). When determining the shortest path in the graph, our algorithm is not conscious of the time of day. Hence, if such a path is selected during the day, the algorithm would be unable to find the relevant trips as these do not exist other than at night. Therefore, we decided filter out night buses and trams from our data when determining the schedule and as a result these are not included in our planner.

- Moreover another limitation of our algorithm is forcing the user to take a bus/tram/train (if it exhists) between two stations even if the distance between the 2 stations is lower then 0.4km. A better way to tackle the problem would have been to allow walking possibilities between any 2 station (knowing their distance is less than 0.4km) but we have chosen to implement a greedy algorithm that does not allow this possibility as we were worried about the complexity of the algorithm becoming too high. We could have also used heuristics to favor some trip_id's that are known to be fast and reliable (a bit like google map does it with A*). However, it would have taken a lot of (too much) data exploration to know which heuristic to implement so we chose not to.